# 35 장. 객체의 사용 관계 이해하기

## 35.1. 사용 관계(Association)

In [1]:
class Master:
    def __init__(self, name):
        self._name = name
        self.Name = Slave()
        
class Slave:
    def get_name(self, other):
        return other._name

In [2]:
m = Master('고요한')

In [3]:
m.Name.get_name(m)

'고요한'

## 35.2. 결합 관계(Aggregation)

In [5]:
class Balance(object):
    def __init__(self, account_code, money=0):
        self.account_code = account_code
        self.balance = money
        
    def set_balance(self, money):
        self.balance += money

In [7]:
class Agreement(object):
    def __init__(self, money, b):
        self.acc_balance = money
        self.b = b
        self.b.set_balance(money)
        
    def __del__(self):
        self.b.set_balance(-self.acc_balance)
        del self

In [8]:
b = Balance('1234')

In [9]:
a = Agreement(100, b)

In [10]:
a.__dict__

{'acc_balance': 100, 'b': <__main__.Balance at 0x7f87b8282130>}

In [11]:
b.__dict__

{'account_code': '1234', 'balance': 100}

In [12]:
del a

In [13]:
b

In [14]:
b.__dict__

{'account_code': '1234', 'balance': 0}

## 35.3. 조합 관계(Composition)

In [25]:
class Balance(object):
    def __init__(self, money=0):
        self._bal = 0
        self.set_balance(money)
        
    def set_balance(self, money):
        self._bal += money

In [26]:
class Agreement(object):
    def __init__(self, acc_no, money):
        self.acc_no = acc_no
        self.balance = Balance(money)
        
    def get_balance(self):
        return self.balance._bal

In [27]:
a = Agreement('1234', 1000)

In [28]:
a.__dict__

{'acc_no': '1234', 'balance': <__main__.Balance at 0x7f87d82b73a0>}

In [29]:
a.get_balance()

1000

In [30]:
del a

In [31]:
a.get_balance()

NameError: name 'a' is not defined

## 35.4. 의존 관계(Dependency)

In [38]:
class Balance(object):
    
    def __init__(self, money=0):
        self._bal = 0
        
    def get_balance(self):
        return self._bal
    
    def set_balance(self, acc_no, money):
        if not hasattr(self, '_acc_no'):
            self._acc_no = acc_no
        self._bal += money
        
    def __del__(self):
        del self

In [39]:
class Agreement(object):
    
    def __init__(self, acc_no, b, money):
        self.acc_no = acc_no
        self.set_balance(b, money)
        
    def get_balance(self, b):
        return b.get_balance()
    
    def set_balance(self, b, money):
        b.set_balance(self.acc_no, money)
        
    def __del__(self):
        del self

In [40]:
b = Balance()

In [41]:
a = Agreement('1234', b, 1000)

In [42]:
a.__dict__

{'acc_no': '1234'}

In [43]:
b.__dict__

{'_bal': 1000, '_acc_no': '1234'}

In [45]:
a.get_balance(b)

1000

In [46]:
del a

In [48]:
a.get_balance(b)

NameError: name 'a' is not defined

In [49]:
b.__dict__

{'_bal': 1000, '_acc_no': '1234'}

In [50]:
del b

In [51]:
b

NameError: name 'b' is not defined

## 35.5. 옵저버(Observer) 패턴 알아보기

In [52]:
from abc import ABC, abstractmethod

In [53]:
from random import randrange

In [56]:
class StateMachine(ABC):
    
    @abstractmethod
    def attach(self, observer):
        pass
    
    @abstractmethod
    def detach(self, observer):
        pass
    
    @abstractmethod
    def notify(self):
        pass